<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/ALZHEIMER_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC  # Import Support Vector Machine
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import shutil
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam

In [3]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the dataset folder
dataset_path = '/content/drive/MyDrive/archive1'  # Replace with your actual folder path

# Verify the folder structure by listing files
import os
print("Dataset folders:", os.listdir(dataset_path))  # Should show 'train' and 'test' folders

# Paths to train and test folders
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

# Verify train and test folders
print("Train folder contents:", os.listdir(train_path))
print("Test folder contents:", os.listdir(test_path))

Mounted at /content/drive
Dataset folders: ['train', 'test']
Train folder contents: ['NonDemented', 'Demented']
Test folder contents: ['Demented', 'NonDemented']


In [4]:
# Step 2: Define image dimensions and batch size
image_size = (224, 224)
batch_size = 32

# Data augmentation setup for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,           # Normalize pixel values to [0, 1]
    rotation_range=30,           # Random rotation up to 30 degrees
    width_shift_range=0.2,       # Horizontal shift
    height_shift_range=0.2,      # Vertical shift
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Random zoom
    horizontal_flip=True,        # Flip images horizontally
    fill_mode='nearest'          # Fill missing pixels after transformation
)

# Data generator for the test set (without augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_path,                   # Use the train folder path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',          # For binary classification
    color_mode='rgb',
    shuffle=True
)

# Load and preprocess testing data
test_generator = test_datagen.flow_from_directory(
    test_path,                    # Use the test folder path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

Found 4861 images belonging to 2 classes.
Found 1603 images belonging to 2 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# Import the Rescaling layer correctly
from tensorflow.keras.layers import Rescaling  # Or tf.keras.layers.experimental.preprocessing.Rescaling for older versions

# ... rest of your code ...

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten

In [8]:
# CNN-LSTM Architecture
model = Sequential()

# Define input shape: 1 time step, 128x128 image with 3 channels
input_shape = (1, 128, 128, 3)

# TimeDistributed Conv2D Layer 1
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'), input_shape=input_shape))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# TimeDistributed Conv2D Layer 2
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# Flatten the output of CNN layers
model.add(TimeDistributed(Flatten()))

# Reshape to LSTM-compatible shape
model.add(Reshape((1, -1)))  # Reshape to (time_steps, features)

# LSTM Layer
model.add(LSTM(100, activation='tanh'))

# Dense Layer (Output)
model.add(Dense(2, activation='sigmoid'))

optimizer = Adam(learning_rate=0.0001)  # Learning rate 0.0001
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the Model
history = model.fit(
    train_generator,
    epochs=25,  # Changed to 100 epochs
    validation_data=test_generator,
    verbose=1
)

# Step 5: Save the Trained Model
model.save('/content/drive/MyDrive/alzheimers_cnn_model.h5')

# Model Summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 224, 224, 3), dtype=float32). Expected shape (None, 1, 128, 128, 3), but input has incompatible shape (None, 224, 224, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • training=True
  • mask=None